In [5]:
#Update your token
STUDENT_TOKEN = 'EUGENIOMARCHIORI'

Server communication game and game_loop.

In [6]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mwinner\033[0m' if game.player == player else '\033[91mloser\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Random solver

In [7]:
def random_solver(game):
  actions = game.actions()
  return actions[0]


**Our heuristic:**<br>
It evaluates the current board state, without predicting future moves, based on the number and positions of potential moves available to both the AI and its opponent. This heuristic score guides the AI's decision-making in the minimax algorithm, influencing move selection to maximize its next move.

In [78]:
def calculate_move_value(board, player, move):
  i, j = move
  value = 0
  # Limiting our own moves good, so let's check the surrounding spaces of our moves
  # to see if we can reduce our options
  if player == 'V':
    # Check left and right
    if j > 0 and board[i][j-1] == '.' and (i == len(board) - 1 or board[i+1][j-1] == '.'):
      value -= 1
    if j < len(board[i]) - 1 and board[i][j+1] == '.' and (i == len(board) - 1 or board[i+1][j+1] == '.'):
      value -= 1
    # Check below
    if i < len(board) - 2 and board[i+2][j] == '.':
      value -= 1

  else:
    # Check above and below
    if i > 0 and board[i-1][j] == '.' and (j == len(board[i]) - 1 or board[i-1][j+1] == '.'):
      value -= 1
    if i < len(board) - 1 and board[i+1][j] == '.' and (j == len(board[i]) - 1 or board[i+1][j+1] == '.'):
      value -= 1

    # Check to the right
    if j < len(board[i]) - 2 and board[i][j+2] == '.':
      value -= 1

  # We will try to avoid playing on edges and corners since if we place on them
  # it limits the opponent's moves, which we dont' want - we want to limit our moves
  if i == 0 or i == len(board) - 1 or j == 0 or j == len(board[i]) - 1:
    value -= 1

  return value


# ---

def static_evaluation(game):
  ai_is_vertical = game.player == 'V'

  # If we are 'V', we go first, check if the board is empty, if so we only care about our agent's score
  if ai_is_vertical and all(cell == '.' for row in game.get_board() for cell in row):
      ai_moves = game.actions()
      return -len(ai_moves)

  # AI score (our agent)
  ai_moves = game.actions()
  ai_score = sum(calculate_move_value(game.get_board(), game.player, move) for move in ai_moves)

  # Opponent's score
  # Temporarily switch to the opponent
  game.player = game.other_player()
  opponent_moves = game.actions()
  opponent_score = sum(calculate_move_value(game.get_board(), game.player, move) for move in opponent_moves)

  # Restore the original player
  game.player = game.other_player()

  return opponent_score - ai_score

# ---
# ---
# ---

def make_move(game, move):
  i, j = move
  player = game.player

  # Save the board before we make the move
  previous_board = deepcopy(game.get_board())
  previous_player = game.player

  if player == 'V':
    game.get_board()[i][j] = 'V'
    game.get_board()[i+1][j] = 'V'
  else:
    game.get_board()[i][j] = 'H'
    game.get_board()[i][j+1] = 'H'

  return previous_board, previous_player

# ---

def undo_move(game, previous_board, previous_player):
  game.board = previous_board
  game.player = previous_player

# ---

def is_game_over(game):
  return len(game.actions()) == 0

# ---

def minimax(game, depth, alpha, beta, maximizing_player):
  # ?
  if depth == 0 or is_game_over(game):
    return static_evaluation(game)

  if maximizing_player:
    max_eval = float('-inf')
    for move in game.actions():
      previous_board, previous_player = make_move(game, move)
      eval = minimax(game, depth - 1, alpha, beta, False)
      undo_move(game, previous_board, previous_player)
      max_eval = max(max_eval, eval)
      alpha = max(alpha, eval)
      if beta <= alpha:
        break
    return max_eval
  else:
    min_eval = float('inf')
    for move in game.actions():
      previous_board, previous_player = make_move(game, move)
      eval = minimax(game, depth - 1, alpha, beta, True)
      undo_move(game, previous_board, previous_player)
      min_eval = min(min_eval, eval)
      beta = min(beta, eval)
      if beta <= alpha:
        break
    return min_eval

Agent heuristic solver

In [76]:
def ai_think(game):
  ai_player = game.player
  ai_is_vertical = (ai_player == 'V')

  # Check if we are max or min our score based on our side
  best_score = float('-inf') if ai_is_vertical else float('inf')
  best_move = None

  for move in game.actions():
    previous_board, previous_player = make_move(game, move)
    score = minimax(game, depth=3, alpha=float('-inf'), beta=float('inf'), maximizing_player=ai_is_vertical)
    undo_move(game, previous_board, previous_player)
    # Now we check the score of that move, based on whether we are maximizing or minimizing
    if ai_is_vertical:
      if score > best_score:
        best_score = score
        best_move = move
    else:
      if score < best_score:
        best_score = score
        best_move = move

  return best_move

Game specific code

In [62]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Domineering(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    actions = []
    board = self.get_board()
    player = self.player

    for row in range(len(board)):
      for col in range(len(board[row])):
        if board[row][col] == '.':
          if player == 'H' and col + 1 < len(board[row]) and board[row][col + 1] == '.':
            actions.append((row, col))
          elif player == 'V' and row + 1 < len(board) and board[row + 1][col] == '.':
            actions.append((row, col))
    return actions

  def print_game(self):
    for row in self.get_board():
      print(row)

  def other_player(self):
    if self.player == 'V': return 'H'
    if self.player == 'H': return 'V'

In [80]:
# game_loop(random_solver, Domineering, 'domineering', multi_player=False, id=None)
game_loop(ai_think, Domineering, 'domineering', multi_player=False, id=None)

Creating new game...
Joining game with id: 7488
Playing as H
Waiting for the other player to join...
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['H', 'H', '.', 'V', '.', '.']
['.', '.', '.', 'V', '.', '.']
['.', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['H', 'H', '.', 'V', '.', '.']
['H', 'H', '.', 'V', '.', '.']
['.', '.', '.', '.', '.', '.']
['V', '.', '.', '.', '.', 'V']
['V', '.', '.', '.', '.', 'V']
['.', '.', '.', '.', '.', '.']
Making next move...
b'Valid move'
['H', 'H', '.', 'V', '.', '.']
['H', 'H', '.', 'V', '.', '.']
['H', 'H', '.', '.', '.', '.']
['V', '.', '.', '.', '.', 'V']
['V', '.', '.', '.', 'V', 'V']
['.', '.', '.', '.', 'V', '.']
Making next move...
b'Valid move'
['H', 'H', '.', 'V'